<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=e4a6318d6c67a6bcb8383776a00d35650f15aade4872aa0720cc0218810c3512
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-14 12:42:01
-------------------
qualified stocks: 87
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  3.00 K
Current:  1.45 C
-------------------
Today PnL: -60.16 K (-0.41%)
Current PnL: -20.39 L (-13.36%)
CY Booked + Current PnL: -6.87 L (-4.5%)
-------------------
Total profit:  1.81 L
Total loss:  -22.21 L
-------------------
Total Booked + Current PnL: 20.11 L (16.01%)
Total Booked PnL: 40.50 L (32.24%)
Curr Year Booked PnL: 13.52 L (9.32%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.36 C
Est FTT PnL: 91.28 L (62.94%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.34%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.00,101.71,51.0,M-SC,2.69,86912.0,-13865.0,13958.0,-0.08,-13.76,16.06,0.09,253.0,-0.99,0.60,14.18,OX40N,NTT,DURABLES
2,ABBOTINDIA,35195.00,-15.75,44.0,X-MC,3.38,87990.0,-2688.0,17598.0,-0.81,-2.96,20.00,16.44,63.0,-0.15,0.61,16.61,X40,ATH,PHARMA
39,INDIGOPNTS,1408.00,133.26,80.0,M-SC,4.36,152830.0,-21729.0,21763.0,2.49,-12.45,14.24,0.02,234.0,-1.00,1.05,32.94,OX40N,NTT,PAINTS
43,ITC,452.00,-40.74,45.0,X-LC,0.88,198583.0,-1555.0,22440.0,-0.22,-0.78,11.30,10.44,5.0,-0.07,1.37,4.74,X40,NTT,FMCG
35,ICICIGI,2252.93,-16.02,54.0,X-MC,7.51,186763.0,12443.0,22766.0,-1.01,7.14,12.19,20.19,68.0,0.55,1.29,23.33,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,STARHEALTH,761.00,37.49,71.0,H-SC,15.22,283315.0,2192.0,130665.0,4.66,0.78,46.12,47.26,171.0,0.02,1.95,51.79,XY24,NTT,INSURANCE
6,ASIANPAINT,3465.66,0.21,86.0,X-LC,8.01,254918.0,3150.0,50066.0,4.59,1.25,19.64,21.13,31.0,0.06,1.76,37.12,X40,ATH,PAINTS
13,BERGEPAINT,680.00,-4.44,70.0,X-MC,1.62,233685.0,6346.0,41713.0,3.02,2.79,17.85,21.14,74.0,0.15,1.61,31.72,XY24,NTT,PAINTS
56,RAJESHEXPO,518.00,1705.56,57.0,L-SC,3.46,49256.0,-87921.0,88016.0,2.99,-64.09,178.69,0.07,268.0,-1.00,0.34,22.50,OX40N,NTT,JEWELLERY
39,INDIGOPNTS,1408.00,133.26,80.0,M-SC,4.36,152830.0,-21729.0,21763.0,2.49,-12.45,14.24,0.02,234.0,-1.00,1.05,32.94,OX40N,NTT,PAINTS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,REPCOHOME,880.00,-55.29,53.0,H-SC,9.48,258200.0,-28305.0,285647.0,-4.95,-9.88,110.63,89.82,160.0,-0.10,1.78,36.61,XY24,NTT,FINANCE
41,INFY,2275.00,-20.67,52.0,X-LC,4.39,318754.0,5866.0,165816.0,-3.56,1.87,52.02,54.87,2.0,0.04,2.20,10.31,X40,BTT,IT
68,SONACOMS,806.63,-34.90,56.0,M-SC,6.32,84166.0,-16993.0,56997.0,-3.38,-16.80,67.72,39.54,220.0,-0.30,0.58,18.68,AR,ATH,AUTO
59,RELAXO,1176.00,-41.83,37.0,X-SC,3.63,79280.0,-65880.0,141808.0,-2.97,-45.38,178.87,52.31,91.0,-0.46,0.55,6.12,X40N,NTT,FOOTWEAR
48,KPIGREEN,731.05,-12.12,38.0,H-SC,10.17,118994.0,-6303.0,65233.0,-2.82,-5.03,54.82,47.03,154.0,-0.10,0.82,48.99,MH,ATH,POWER


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WHIRLPOOL,2270.00,-41.44,40.0,M-SC,9.17,114706.0,-92.0,94139.0,1.62,-0.08,82.07,81.92,236.0,-0.00,0.79,35.79,XR,NTT,DURABLES
25,FINCABLES,1641.55,-18.93,46.0,M-SC,10.44,146842.0,-802.0,160131.0,-0.40,-0.54,109.05,107.91,216.0,-0.01,1.01,2.44,OX40N,ATH,CABLES
69,STARHEALTH,761.00,37.49,71.0,H-SC,15.22,283315.0,2192.0,130665.0,4.66,0.78,46.12,47.26,171.0,0.02,1.95,51.79,XY24,NTT,INSURANCE
86,ZYDUSLIFE,1286.17,-14.53,37.0,H-MC,5.89,208834.0,-740.0,75410.0,-0.68,-0.35,36.11,35.63,119.0,-0.01,1.44,16.26,AR,ATH,PHARMA
85,WIPRO,420.00,-14.25,51.0,M-LC,5.50,150784.0,-161.0,109620.0,-0.82,-0.11,72.70,72.51,99.0,-0.00,1.04,5.91,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-18.93,46.0,M-SC,10.44,146842.0,-802.0,160131.0,-0.40,-0.54,109.05,107.91,216.0,-0.01,1.01,2.44,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,133.26,80.0,M-SC,4.36,152830.0,-21729.0,21763.0,2.49,-12.45,14.24,0.02,234.0,-1.00,1.05,32.94,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,101.71,51.0,M-SC,2.69,86912.0,-13865.0,13958.0,-0.08,-13.76,16.06,0.09,253.0,-0.99,0.60,14.18,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-65.79,53.0,H-SC,2.41,224658.0,-45009.0,81349.0,1.67,-16.69,36.21,13.47,158.0,-0.55,1.55,14.23,XY24,NTT,PAINTS
44,JCHAC,2282.00,19986.96,65.0,M-SC,1.48,98157.0,-29648.0,29634.0,0.19,-23.20,30.19,-0.01,233.0,-1.00,0.68,11.15,OX40N,NTT,AC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,STARHEALTH,761.00,37.49,71.0,H-SC,15.22,283315.0,2192.0,130665.0,4.66,0.78,46.12,47.26,171.0,0.02,1.95,51.79,XY24,NTT,INSURANCE
62,SAMMAANCAP,326.00,17.65,53.0,M-SC,36.04,158976.0,9756.0,134430.0,-0.19,6.54,84.56,96.62,245.0,0.07,1.10,72.05,XY25,NTT,FINANCE
38,INDIAMART,4810.62,-47.77,52.0,H-SC,4.74,130242.0,6906.0,124720.0,-1.88,5.60,95.76,106.72,139.0,0.06,0.90,30.29,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-47.77,52.0,H-SC,4.74,130242.0,6906.0,124720.0,-1.88,5.60,95.76,106.72,139.0,0.06,0.90,30.29,AR,ATH,MISC
84,WHIRLPOOL,2270.00,-41.44,40.0,M-SC,9.17,114706.0,-92.0,94139.0,1.62,-0.08,82.07,81.92,236.0,-0.00,0.79,35.79,XR,NTT,DURABLES
85,WIPRO,420.00,-14.25,51.0,M-LC,5.50,150784.0,-161.0,109620.0,-0.82,-0.11,72.70,72.51,99.0,-0.00,1.04,5.91,XR,NTT,IT
86,ZYDUSLIFE,1286.17,-14.53,37.0,H-MC,5.89,208834.0,-740.0,75410.0,-0.68,-0.35,36.11,35.63,119.0,-0.01,1.44,16.26,AR,ATH,PHARMA
25,FINCABLES,1641.55,-18.93,46.0,M-SC,10.44,146842.0,-802.0,160131.0,-0.40,-0.54,109.05,107.91,216.0,-0.01,1.01,2.44,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,12.06,27.0,X-SC,9.01,85490.0,-43180.0,88482.0,-2.60,-33.56,103.50,35.20,92.0,-0.49,0.59,0.00,X40,NTT,FOOTWEAR
27,GLAXO,3466.20,-19.47,29.0,X-MC,5.34,199768.0,484.0,77530.0,-0.38,0.24,38.81,39.15,60.0,0.01,1.38,29.32,X40,ATH,PHARMA
26,GILLETTE,11206.78,-17.51,29.0,X-SC,5.76,254820.0,924.0,81390.0,-0.18,0.36,31.94,32.42,85.0,0.01,1.76,15.52,X40,ATH,FMCG
55,QUESS,986.00,-53.17,29.0,X-SC,37.31,46700.0,-18306.0,169236.0,-2.81,-28.16,362.39,232.18,83.0,-0.11,0.32,0.00,XY24,NTT,MISC
46,JSWINFRA,342.00,-19.06,31.0,X-MC,5.37,189472.0,-10875.0,41381.0,-0.23,-5.43,21.84,15.23,66.0,-0.26,1.31,25.07,X40N,NTT,REALTY


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,1065.00,-16.97,37.0,X-LC,0.08,256465.0,-19945.0,441120.0,-1.61,-7.22,172.00,152.37,3.0,-0.05,1.77,0.18,XY24,NTT,AUTO
36,ICICIPRULI,790.00,-19.77,70.0,X-MC,0.84,185454.0,8716.0,46011.0,1.32,4.93,24.81,30.97,75.0,0.19,1.28,18.11,X40,ATH,INSURANCE
43,ITC,452.00,-40.74,45.0,X-LC,0.88,198583.0,-1555.0,22440.0,-0.22,-0.78,11.30,10.44,5.0,-0.07,1.37,4.74,X40,NTT,FMCG
34,HONAUT,58357.33,-24.98,36.0,X-SC,0.97,106620.0,-21318.0,68450.0,0.13,-16.66,64.20,36.84,90.0,-0.31,0.74,9.15,X40N,ATH,ELECTRICAL
16,CAMS,4762.00,-1.47,57.0,X-SC,1.44,260436.0,8580.0,53858.0,-0.45,3.41,20.68,24.79,86.0,0.16,1.80,27.33,X40N,NTT,MISC


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,12.06,27.0,X-SC,9.01,85490.0,-43180.0,88482.0,-2.60,-33.56,103.50,35.20,92.0,-0.49,0.59,0.00,X40,NTT,FOOTWEAR
55,QUESS,986.00,-53.17,29.0,X-SC,37.31,46700.0,-18306.0,169236.0,-2.81,-28.16,362.39,232.18,83.0,-0.11,0.32,0.00,XY24,NTT,MISC
76,TMPV,1065.00,-16.97,37.0,X-LC,0.08,256465.0,-19945.0,441120.0,-1.61,-7.22,172.00,152.37,3.0,-0.05,1.77,0.18,XY24,NTT,AUTO
53,PAGEIND,51769.93,-28.58,37.0,X-MC,7.64,158780.0,-4960.0,48301.0,-2.52,-3.03,30.42,26.47,55.0,-0.10,1.10,0.60,X40,ATH,APPARELS
20,COLPAL,3726.84,-1.90,40.0,X-MC,6.98,216700.0,-46665.0,155981.0,-0.48,-17.72,71.98,41.51,61.0,-0.30,1.49,1.69,XY25,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,986.00,-53.17,29.0,X-SC,37.31,46700.0,-18306.0,169236.0,-2.81,-28.16,362.39,232.18,83.0,-0.11,0.32,0.00,XY24,NTT,MISC
59,RELAXO,1176.00,-41.83,37.0,X-SC,3.63,79280.0,-65880.0,141808.0,-2.97,-45.38,178.87,52.31,91.0,-0.46,0.55,6.12,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,12.06,27.0,X-SC,9.01,85490.0,-43180.0,88482.0,-2.60,-33.56,103.50,35.20,92.0,-0.49,0.59,0.00,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-15.75,44.0,X-MC,3.38,87990.0,-2688.0,17598.0,-0.81,-2.96,20.00,16.44,63.0,-0.15,0.61,16.61,X40,ATH,PHARMA
34,HONAUT,58357.33,-24.98,36.0,X-SC,0.97,106620.0,-21318.0,68450.0,0.13,-16.66,64.20,36.84,90.0,-0.31,0.74,9.15,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,-26.86,58.0,X-LC,8.79,290780.0,-55176.0,121866.0,-1.23,-15.95,41.91,19.28,1.0,-0.45,2.01,7.50,X40,ATH,IT
41,INFY,2275.00,-20.67,52.0,X-LC,4.39,318754.0,5866.0,165816.0,-3.56,1.87,52.02,54.87,2.0,0.04,2.20,10.31,X40,BTT,IT
76,TMPV,1065.00,-16.97,37.0,X-LC,0.08,256465.0,-19945.0,441120.0,-1.61,-7.22,172.00,152.37,3.0,-0.05,1.77,0.18,XY24,NTT,AUTO
82,VBL,671.64,-18.42,46.0,X-LC,5.86,292236.0,-23606.0,136270.0,-0.27,-7.47,46.63,35.67,4.0,-0.17,2.02,5.34,X40N,ATH,FMCG
43,ITC,452.00,-40.74,45.0,X-LC,0.88,198583.0,-1555.0,22440.0,-0.22,-0.78,11.30,10.44,5.0,-0.07,1.37,4.74,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-47.69,44.0,L-SC,25.83,77376.0,-36173.0,76223.0,-0.37,-31.86,98.51,35.27,270.0,-0.47,0.53,89.38,XR,NTT,HOTELS
7,ASIANTILES,137.00,7555.56,64.0,L-SC,15.83,83995.0,-9815.0,86431.0,-1.65,-10.46,102.90,81.67,271.0,-0.11,0.58,62.66,XR,NTT,CERAMICS
51,MASFIN,398.61,-18.25,52.0,H-SC,7.53,92700.0,-5280.0,26883.0,0.32,-5.39,29.00,22.05,164.0,-0.20,0.64,34.75,XR,ATH,FINANCE
38,INDIAMART,4810.62,-47.77,52.0,H-SC,4.74,130242.0,6906.0,124720.0,-1.88,5.60,95.76,106.72,139.0,0.06,0.90,30.29,AR,ATH,MISC
80,VAIBHAVGBL,521.00,73.00,61.0,H-SC,7.56,149911.0,-32864.0,143930.0,0.34,-17.98,96.01,60.77,159.0,-0.23,1.03,35.01,XR,NTT,JEWELLERY


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANPAINT,3465.66,0.21,86.0,X-LC,8.01,254918.0,3150.0,50066.0,4.59,1.25,19.64,21.13,31.0,0.06,1.76,37.12,X40,ATH,PAINTS
39,INDIGOPNTS,1408.00,133.26,80.0,M-SC,4.36,152830.0,-21729.0,21763.0,2.49,-12.45,14.24,0.02,234.0,-1.00,1.05,32.94,OX40N,NTT,PAINTS
80,VAIBHAVGBL,521.00,73.00,61.0,H-SC,7.56,149911.0,-32864.0,143930.0,0.34,-17.98,96.01,60.77,159.0,-0.23,1.03,35.01,XR,NTT,JEWELLERY
7,ASIANTILES,137.00,7555.56,64.0,L-SC,15.83,83995.0,-9815.0,86431.0,-1.65,-10.46,102.90,81.67,271.0,-0.11,0.58,62.66,XR,NTT,CERAMICS
40,INDUSINDBK,1800.00,-702.61,74.0,L-MC,6.73,48803.0,-28404.0,53796.0,-0.95,-36.79,110.23,32.89,259.0,-0.53,0.34,34.40,XR,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.59
1,25,44.48
2,50,75.70


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    47.56
MC    27.75
LC    24.69
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.50
X40      20.53
X40N     12.25
XY25     10.05
XR        9.57
AR        9.00
OX40N     7.48
X200      1.80
SR        1.00
MH        0.82
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.24
X-MC    21.38
X-LC    19.80
M-SC    13.30
X-SC     8.57
H-MC     4.66
H-LC     2.87
L-SC     1.45
M-MC     1.37
M-LC     1.04
L-LC     0.98
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.61,-4.42,38.19
IT,12.47,-16.34,78.62
FINANCE,11.94,-9.17,58.65
MISC,7.55,-18.84,78.79
PAINTS,5.97,-6.61,22.50
ELECTRICAL,5.56,-12.06,51.79
INSURANCE,4.52,3.56,30.42
AUTO,3.45,-14.88,125.03
PHARMA,3.43,-0.59,34.34


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3317548.0,20
XR,1268592.0,13
AR,1266894.0,10
X40,995391.0,14
OX40N,706796.0,10
X40N,694490.0,9
XY25,476271.0,7
SR,274035.0,2
MH,65233.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3486758.0,25
M-SC,1423667.0,16
X-LC,1197570.0,11
X-MC,1142985.0,15
X-SC,834468.0,8
H-MC,403067.0,3
L-SC,250670.0,3
H-LC,142081.0,2
M-LC,109620.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1166824.0      6
           AR         842802.0      5
           XR         799326.0      7
M-SC       XY24       739842.0      6
X-LC       XY24       503728.0      2
           X40        413074.0      5
X-MC       X40        412445.0      7
X-SC       XY24       375632.0      2
X-MC       XY24       347784.0      3
H-SC       OX40N      338538.0      4
X-SC       X40N       288964.0      4
M-SC       OX40N      280242.0      5
H-SC       SR         274035.0      2
H-MC       AR         219329.0      2
X-LC       X40N       204051.0      2
X-MC       X40N       201475.0      3
H-MC       XY24       183738.0      1
X-MC       XY25       181281.0      2
X-SC       X40        169872.0      2
L-SC       XR         162654.0      2
M-SC       XR         143196.0      2
           XY25       134430.0      1
           AR         125957.0      2
M-LC       XR         109620.0      1
L-SC       OX40N       88016.0      1
H-LC       AR          78806.0      1
X-LC       XY25        76717.0      2
H-SC       MH          65233.0      1
H-LC       X200        63275.0      1
L-MC       XR          53796.0      1
M-MC       XY25        47077.0      1
L-LC       XY25        36766.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 40.0 seconds
